# Tercera entrega - Trabajo Integrador - **Grupo 17 5k10**
##  **Integrantes:** Veggiani Franco, Diaz Juan Ignacio

### Enunciado:
En esta entrega se espera:
1. Que cada grupo defina claramente el objetivo predictivo del proyecto, identificando con precisión la variable objetivo y el tipo de problema (clasificación, regresión, etc.).
2. El análisis debe incluir la estrategia de partición del conjunto de datos (entrenamiento/test o validación cruzada), y la construcción de un pipeline completo de Scikit-learn que integre todas las etapas necesarias: preprocesamiento, modelado y evaluación.
3. Deberán comparar al menos tres modelos distintos y justificar cuál se ajusta mejor al problema.
4. Luego, deberán realizar un proceso de búsqueda y ajuste de hiperparámetros sobre el modelo elegido, utilizando técnicas como GridSearchCV o RandomizedSearchCV.
5. Se espera una evaluación rigurosa con métricas adecuadas (accuracy, F1, RMSE, etc., según el caso), así como un diagnóstico de overfitting o underfitting y posibles acciones para mitigarlos.

La entrega debe presentarse como un Jupyter Notebook claro, bien estructurado y reproducible, y los integrantes del grupo deben poder explicar oralmente las decisiones tomadas durante todo el proceso de modelado.

## 0. Definición del pipeline

In [51]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector

import numpy as np

In [52]:
# ============================================================================
# ALTERNATIVA A LAGS: FEATURES AGREGADAS HISTÓRICAS
# ============================================================================
# En lugar de usar lag_1, lag_7, etc., usamos estadísticas agregadas
# por grupo que se calculan UNA VEZ y se guardan como "lookup table"
# ============================================================================

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class HistoricalProfileEncoder(BaseEstimator, TransformerMixin):
    """
    Codifica el "perfil histórico" de cada línea-municipio-día_semana.

    En ENTRENAMIENTO:
    - Calcula estadísticas agregadas por grupo

    En PREDICCIÓN:
    - Usa las estadísticas pre-calculadas (NO necesita historial reciente)

    Ejemplo:
    --------
    Para predecir la línea 740 en San Miguel un Lunes:
    - NO usa cuántos pasajeros hubo ayer (lag_1) ✗
    - SÍ usa "los lunes en esta línea suelen haber X pasajeros" ✓
    """

    def __init__(self, group_cols=['linea', 'municipio', 'dia_semana']):
        self.group_cols = group_cols
        self.profiles = {}
        self.global_stats = {}

    def fit(self, X, y):
        """Calcula perfiles históricos desde los datos de entrenamiento"""

        df = X.copy()
        df['cantidad'] = y

        print("\n📊 Calculando perfiles históricos...")

        # 1. Estadísticas por grupo (línea + municipio + día de semana)
        group_stats = df.groupby(self.group_cols)['cantidad'].agg([
            'mean',    # Promedio histórico
            'std',     # Variabilidad
            'median',  # Mediana (robusto a outliers)
            'min',     # Mínimo histórico
            'max',     # Máximo histórico
            'count'    # Cantidad de observaciones
        ]).reset_index()

        self.profiles['main'] = group_stats

        # 2. Estadísticas por línea + municipio (sin día de semana)
        line_muni_stats = df.groupby(['linea', 'municipio'])['cantidad'].agg([
            'mean', 'std', 'median'
        ]).reset_index()
        line_muni_stats.columns = ['linea', 'municipio', 'lm_mean', 'lm_std', 'lm_median']
        self.profiles['line_muni'] = line_muni_stats

        # 3. Estadísticas por línea + día de semana
        line_day_stats = df.groupby(['linea', 'dia_semana'])['cantidad'].agg([
            'mean', 'std'
        ]).reset_index()
        line_day_stats.columns = ['linea', 'dia_semana', 'ld_mean', 'ld_std']
        self.profiles['line_day'] = line_day_stats

        # 4. Estadísticas por municipio + día de semana
        muni_day_stats = df.groupby(['municipio', 'dia_semana'])['cantidad'].agg([
            'mean', 'std'
        ]).reset_index()
        muni_day_stats.columns = ['municipio', 'dia_semana', 'md_mean', 'md_std']
        self.profiles['muni_day'] = muni_day_stats

        # 5. Estadísticas globales (fallback)
        self.global_stats = {
            'mean': df['cantidad'].mean(),
            'std': df['cantidad'].std(),
            'median': df['cantidad'].median()
        }

        print(f"   ✓ {len(group_stats)} perfiles únicos creados")
        print(f"   ✓ Cobertura: {(group_stats['count'].sum() / len(df)) * 100:.1f}%")

        return self

    def transform(self, X):
        """Agrega features basadas en perfiles históricos"""

        X_new = X.copy()

        # Merge con estadísticas principales
        X_new = X_new.merge(
            self.profiles['main'],
            on=self.group_cols,
            how='left'
        )

        # Merge con estadísticas línea-municipio
        X_new = X_new.merge(
            self.profiles['line_muni'],
            on=['linea', 'municipio'],
            how='left'
        )

        # Merge con estadísticas línea-día
        X_new = X_new.merge(
            self.profiles['line_day'],
            on=['linea', 'dia_semana'],
            how='left'
        )

        # Merge con estadísticas municipio-día
        X_new = X_new.merge(
            self.profiles['muni_day'],
            on=['municipio', 'dia_semana'],
            how='left'
        )

        # Rellenar valores faltantes con estadísticas globales
        fill_cols = ['mean', 'std', 'median', 'lm_mean', 'lm_std', 'lm_median',
                     'ld_mean', 'ld_std', 'md_mean', 'md_std']

        for col in fill_cols:
            if col in X_new.columns:
                base_stat = 'mean' if 'mean' in col else 'std' if 'std' in col else 'median'
                X_new[col].fillna(self.global_stats[base_stat], inplace=True)

        # Features derivadas
        X_new['volatility'] = X_new['std'] / (X_new['mean'] + 1)  # Coeficiente de variación
        X_new['normalized_demand'] = X_new['mean'] / (X_new['lm_mean'] + 1)  # Demanda relativa

        return X_new

In [53]:
class WeatherImpactEncoder(BaseEstimator, TransformerMixin):
    """
    Codifica cómo el clima afecta históricamente a cada línea-municipio.

    Ejemplo:
    --------
    - Algunas líneas son muy sensibles a lluvia (zonas sin alternativas)
    - Otras no tanto (zonas céntricas con múltiples opciones)
    """

    def __init__(self):
        self.weather_impacts = {}

    def fit(self, X, y):
        """Calcula sensibilidad al clima por grupo"""

        df = X.copy()
        df['cantidad'] = y

        print("\n🌦️ Calculando impacto del clima...")

        for group in df.groupby(['linea', 'municipio']):
            key = group[0]
            data = group[1]

            if len(data) < 10:  # Muy pocos datos
                continue

            # Correlación entre lluvia y demanda
            rain_corr = data[['precip', 'cantidad']].corr().iloc[0, 1]

            # Diferencia de demanda en días lluviosos vs secos
            rainy_days = data[data['precip'] > 5]['cantidad'].mean()
            dry_days = data[data['precip'] <= 5]['cantidad'].mean()
            rain_impact = (rainy_days - dry_days) / dry_days if dry_days > 0 else 0

            # Sensibilidad a temperatura
            temp_corr = data[['t_med', 'cantidad']].corr().iloc[0, 1]

            self.weather_impacts[key] = {
                'rain_correlation': rain_corr,
                'rain_impact_pct': rain_impact,
                'temp_correlation': temp_corr
            }

        print(f"   ✓ {len(self.weather_impacts)} perfiles climáticos creados")

        return self

    def transform(self, X):
        """Agrega features de impacto climático"""

        X_new = X.copy()

        # Inicializar columnas
        X_new['rain_sensitivity'] = 0.0
        X_new['rain_impact'] = 0.0
        X_new['temp_sensitivity'] = 0.0

        # Aplicar perfiles
        for idx, row in X_new.iterrows():
            key = (row['linea'], row['municipio'])

            if key in self.weather_impacts:
                impacts = self.weather_impacts[key]
                X_new.loc[idx, 'rain_sensitivity'] = impacts['rain_correlation']
                X_new.loc[idx, 'rain_impact'] = impacts['rain_impact_pct']
                X_new.loc[idx, 'temp_sensitivity'] = impacts['temp_correlation']

        # Interacciones clima × sensibilidad
        X_new['adjusted_rain'] = X_new['precip'] * X_new['rain_sensitivity']
        X_new['adjusted_temp'] = X_new['t_med'] * X_new['temp_sensitivity']

        return X_new


In [54]:
class SeasonalityEncoder(BaseEstimator, TransformerMixin):
    """
    Codifica patrones estacionales (mensuales/semanales) por grupo.
    """

    def __init__(self):
        self.seasonal_patterns = {}

    def fit(self, X, y):
        """Calcula factores estacionales"""

        df = X.copy()
        df['cantidad'] = y

        print("\n📅 Calculando patrones estacionales...")

        # Por línea-municipio-mes
        monthly = df.groupby(['linea', 'municipio', 'mes'])['cantidad'].mean()

        # Normalizar por promedio anual de cada grupo
        for (linea, municipio) in df.groupby(['linea', 'municipio']).groups.keys():
            subset = monthly.loc[linea, municipio]
            if len(subset) > 0:
                annual_avg = subset.mean()
                if annual_avg > 0:
                    for mes in subset.index:
                        key = (linea, municipio, mes)
                        self.seasonal_patterns[key] = subset[mes] / annual_avg

        print(f"   ✓ {len(self.seasonal_patterns)} patrones estacionales")

        return self

    def transform(self, X):
        """Agrega factor estacional"""

        X_new = X.copy()
        X_new['seasonal_factor'] = 1.0

        for idx, row in X_new.iterrows():
            key = (row['linea'], row['municipio'], row['mes'])
            if key in self.seasonal_patterns:
                X_new.loc[idx, 'seasonal_factor'] = self.seasonal_patterns[key]

        return X_new


In [55]:
class DateSorter(BaseEstimator, TransformerMixin):
    """
    Convierte y ordena por fecha. Debe ser el primer paso del pipeline.
    """
    def __init__(self, date_column='fecha'):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_any_dtype(X[self.date_column]):
            X[self.date_column] = pd.to_datetime(X[self.date_column])

        # Ordenar por fecha
        X = X.sort_values(self.date_column).reset_index(drop=True)

        return X


In [56]:
class TemporalFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Extrae features temporales de la columna fecha y crea features cíclicas.
    """
    def __init__(self, date_column='fecha'):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # La fecha ya debe estar como datetime gracias a DateSorter

        # Extraer features temporales básicas
        X['dia_semana'] = X[self.date_column].dt.dayofweek
        X['mes'] = X[self.date_column].dt.month
        X['is_weekend'] = X['dia_semana'].apply(lambda x: 1 if x >= 5 else 0)

        # Features cíclicas para mes
        X['mes_sin'] = np.sin(2 * np.pi * X['mes'] / 12)
        X['mes_cos'] = np.cos(2 * np.pi * X['mes'] / 12)

        # Features cíclicas para día de semana
        X['dia_sin'] = np.sin(2 * np.pi * X['dia_semana'] / 7)
        X['dia_cos'] = np.cos(2 * np.pi * X['dia_semana'] / 7)

        return X


In [57]:
class TemperatureFeatureCreator(BaseEstimator, TransformerMixin):
    """
    Crea features derivadas de temperatura.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Temperatura media
        X['t_med'] = (X['tmax'] + X['tmin']) / 2

        # Amplitud térmica
        X['t_amp'] = X['tmax'] - X['tmin']

        return X

In [58]:
class DropNaRows(BaseEstimator, TransformerMixin):
    """
    Elimina filas que tengan NaN en columnas específicas.
    Pensado para usarse luego de LagFeatureCreator, antes del split X/y.
    """
    def __init__(self, columns=None, how='any'):
        self.columns = columns or []
        self.how = how  # 'any' o 'all'

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if self.columns:
            X = X.dropna(subset=self.columns, how=self.how)
        X = X.reset_index(drop=True)
        return X

In [59]:
class LagFeatureCreator(BaseEstimator, TransformerMixin):
    """
    Crea features de lag y rolling para series temporales.
    Agrupa por línea y municipio para mantener la coherencia temporal.
    """
    def __init__(self, target_col='cantidad', date_col='fecha',
                 group_cols=['linea', 'municipio'],
                 lags=[1, 7, 28],
                 rolling_windows=[7, 28]):
        self.target_col = target_col
        self.date_col = date_col
        self.group_cols = group_cols
        self.lags = lags
        self.rolling_windows = rolling_windows

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Asegurar que fecha sea datetime y ordenar
        if not pd.api.types.is_datetime64_any_dtype(X[self.date_col]):
            X[self.date_col] = pd.to_datetime(X[self.date_col])

        X = X.sort_values(self.date_col).reset_index(drop=True)

        def add_lags_rolls(g):
            g = g.sort_values(self.date_col)

            # Crear lags
            for lag in self.lags:
                g[f'lag_{lag}'] = g[self.target_col].shift(lag)

            # # Crear rolling means
            # for window in self.rolling_windows:
            #     g[f'roll_{window}'] = g[self.target_col].shift(1).rolling(
            #         window=window, min_periods=1
            #     ).mean()

            # Indicadores de lag faltante (opcional)
            for lag in self.lags:
                g[f'has_lag_{lag}'] = g[f'lag_{lag}'].isnull().astype(int)

            return g

        # Aplicar por grupo
        X = X.groupby(self.group_cols, group_keys=False).apply(
            add_lags_rolls
        )

        return X

In [60]:
class DropColumns(BaseEstimator, TransformerMixin):
    """
    Elimina columnas específicas del DataFrame.
    """
    def __init__(self, columns_to_drop=None):
        self.columns_to_drop = columns_to_drop or []

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            return X.drop(columns=self.columns_to_drop, errors='ignore')
        return X


In [61]:
class Winsorizer(BaseEstimator, TransformerMixin):
    """
    Aplica winsorización a columnas específicas para manejar outliers.
    """
    def __init__(self, lower_percentile=0.01, upper_percentile=0.99, columns=None):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile
        self.columns = columns
        self.lower_bounds = {}
        self.upper_bounds = {}

    def fit(self, X, y=None):
        if self.columns is None:
            self.columns = X.columns

        for col in self.columns:
            if col in X.columns:
                self.lower_bounds[col] = np.percentile(
                    X[col].dropna(), self.lower_percentile * 100
                )
                self.upper_bounds[col] = np.percentile(
                    X[col].dropna(), self.upper_percentile * 100
                )
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for col in self.columns:
            if col in X_transformed.columns:
                X_transformed[col] = np.clip(
                    X_transformed[col],
                    self.lower_bounds[col],
                    self.upper_bounds[col]
                )
        return X_transformed

In [62]:
def create_fe_pipeline(
    target_col='cantidad',
    date_col='fecha',
    group_cols=['linea', 'municipio'],
    lags=[28],
    rolling_windows=[7, 28],
    dropna_lag_cols=('cantidad', 'lag_1', 'lag_7', 'lag_28', 'roll_7', 'roll_28')
):
    return Pipeline([
        ("date_sorter", DateSorter(date_column='fecha')),
        ("temporal_features", TemporalFeatureExtractor(date_column='fecha')),
        ("temperature_features", TemperatureFeatureCreator()),
        ("lag_features", LagFeatureCreator(lags=lags, group_cols=group_cols))
    ])


In [63]:
# Recrear el pipeline final sin lambdas
def create_final_preprocessing_pipeline(
    columns_to_drop=['fecha', 'provincia', 'nombre_feriado', 'tipo_transporte', 'tipo_feriado', 'linea', 'empresa'],
    winsorize_cols=['t_med', 'precip', 'tmax', 'tmin']
):
    """
    Versión serializable del pipeline final.
    Usa make_column_selector en lugar de lambdas.
    """
    numeric_pipeline = Pipeline([
        ("winsorizer", Winsorizer(columns=winsorize_cols)),
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", MinMaxScaler())
    ])

    categorical_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    return Pipeline([
        ("drop_columns", DropColumns(columns_to_drop=columns_to_drop)),
        ("column_transform", ColumnTransformer([
            ("num", numeric_pipeline, make_column_selector(dtype_include=["int64", "float64"])),
            ("cat", categorical_pipeline, make_column_selector(dtype_include=["object", "category"]))
        ]))
    ])

## 1. Objetivo predictivo del proyecto

- **Objetivo del proyecto**:  Predecir la cantidad de pasajeros transportados por una línea de colectivo en una fecha determinada, utilizando información contextual como clima, feriados, ubicación y características del servicio.

  - **Variable objetivo**: `cantidad` (número entero que representa la cantidad de pasajeros)

  - **Tipo de problema**: `Regresión`, ya que se busca predecir un valor numérico continuo.

- **Justificación**: La variable `cantidad` no representa clases ni categorías, sino un conteo que varía en función de múltiples factores. Por lo tanto, se trata de un problema de regresión supervisada.



## 2. Análisis del dataset

In [64]:
import pandas as pd
df = pd.read_csv('final_2024-11-04.csv')
print("---- (FILAS, COLUMNAS) -----")
print(df.shape)
print("---- (TIPOS DE DATOS) -----")
print(df.dtypes)
print("---- (ESTADÍSTICOS VARIABLE OBJETIVO) -----")
print(df['cantidad'].describe())
print("---- (NULOS) -----")
print(df.isnull().sum())

# Drop rows with NaN in specified columns
df = df.dropna(subset=['tmax', 'tmin', 'precip', 'viento'], how='any').reset_index(drop=True)

print("---- (FILAS, COLUMNAS DESPUÉS DE DROPEAR NAN) -----")
print(df.shape)
print("---- (NULOS DESPUÉS DE DROPEAR NAN) -----")
print(df.isnull().sum())

---- (FILAS, COLUMNAS) -----
(133204, 14)
---- (TIPOS DE DATOS) -----
fecha               object
empresa             object
linea               object
tipo_transporte     object
provincia           object
municipio           object
cantidad             int64
tmax               float64
tmin               float64
precip             float64
viento             float64
is_feriado           int64
tipo_feriado        object
nombre_feriado      object
dtype: object
---- (ESTADÍSTICOS VARIABLE OBJETIVO) -----
count    133204.000000
mean      14055.787386
std       19946.489535
min           1.000000
25%        2169.000000
50%        6973.000000
75%       17665.000000
max      189636.000000
Name: cantidad, dtype: float64
---- (NULOS) -----
fecha                   0
empresa                 0
linea                   0
tipo_transporte         0
provincia              22
municipio              22
cantidad                0
tmax                 2137
tmin                 2137
precip               2137


###  2.1 Preprocesamiento y pipeline

**MUY IMPORTANTE ESTO** Se obtiene el dia y el mes de la semana, porque la fecha como tal no sirve para el entrenamiento, no se puede generalizar.

Debemos obtener también el número de día en la semana (0 - 6)

In [65]:
import pandas as pd
df = pd.read_csv('final_2024-11-04.csv')
print("---- (FILAS, COLUMNAS) -----")
print(df.shape)
print("---- (TIPOS DE DATOS) -----")
print(df.dtypes)
print("---- (ESTADÍSTICOS VARIABLE OBJETIVO) -----")
print(df['cantidad'].describe())
print("---- (NULOS) -----")
print(df.isnull().sum())

# Drop rows with NaN in specified columns
df = df.dropna(subset=['tmax', 'tmin', 'precip', 'viento'], how='any').reset_index(drop=True)

print("---- (FILAS, COLUMNAS DESPUÉS DE DROPEAR NAN) -----")
print(df.shape)
print("---- (NULOS DESPUÉS DE DROPEAR NAN) -----")
print(df.isnull().sum())

---- (FILAS, COLUMNAS) -----
(133204, 14)
---- (TIPOS DE DATOS) -----
fecha               object
empresa             object
linea               object
tipo_transporte     object
provincia           object
municipio           object
cantidad             int64
tmax               float64
tmin               float64
precip             float64
viento             float64
is_feriado           int64
tipo_feriado        object
nombre_feriado      object
dtype: object
---- (ESTADÍSTICOS VARIABLE OBJETIVO) -----
count    133204.000000
mean      14055.787386
std       19946.489535
min           1.000000
25%        2169.000000
50%        6973.000000
75%       17665.000000
max      189636.000000
Name: cantidad, dtype: float64
---- (NULOS) -----
fecha                   0
empresa                 0
linea                   0
tipo_transporte         0
provincia              22
municipio              22
cantidad                0
tmax                 2137
tmin                 2137
precip               2137


In [66]:
preprocessor = create_fe_pipeline()

In [67]:
df_fe = preprocessor.fit_transform(df)

/tmp/ipykernel_139606/633612192.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  X = X.groupby(self.group_cols, group_keys=False).apply(


In [68]:
# 4) Split temporal 70/15/15 (la columna 'fecha' ya es datetime y está ordenada)
cut_train = df_fe["fecha"].quantile(0.70)
cut_valid = df_fe["fecha"].quantile(0.85)

train_mask = df_fe["fecha"] <= cut_train
valid_mask = (df_fe["fecha"] > cut_train) & (df_fe["fecha"] <= cut_valid)
test_mask  = df_fe["fecha"] > cut_valid

df_train = df_fe[train_mask].copy()
df_valid = df_fe[valid_mask].copy()
df_test  = df_fe[test_mask].copy()

# 5) Separar X e y (recién ahora sacamos la target de X)
X_train, y_train = df_train.drop(columns=["cantidad"]), df_train["cantidad"]
X_valid, y_valid = df_valid.drop(columns=["cantidad"]), df_valid["cantidad"]
X_test,  y_test  = df_test.drop(columns=["cantidad"]),  df_test["cantidad"]

In [69]:
# 5) Preprocesamiento final (ONEHOT/SCALER) recién ahora
final_pre = create_final_preprocessing_pipeline()
X_train_transformed = final_pre.fit_transform(X_train)
X_valid_transformed = final_pre.transform(X_valid)
X_test_transformed  = final_pre.transform(X_test)

## 3. Comparación de modelos

### 3.1 Evaluación con métricas


In [70]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time


In [71]:
def safe_mape(y_true, y_pred, eps=1e-6):
    """Calcula MAPE de forma segura evitando división por cero"""
    denom = np.where(np.abs(y_true) < eps, eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

def calculate_metrics(y_true, y_pred, dataset_name):
    """Calcula todas las métricas relevantes"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = safe_mape(np.array(y_true), np.array(y_pred))

    return {
        'Dataset': dataset_name,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE (%)': mape
    }

In [84]:
print("\n📋 Modelos a comparar:")
print("   1. Linear Regression (Baseline)")
print("   2. Random Forest")
print("   3. Gradient Boosting")
print("   4. XGBoost (con configuración optimizada)")

models = {
    'Linear Regression': LinearRegression(),

    'Random Forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1
    ),

    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    ),

    'XGBoost': XGBRegressor(
        n_estimators=500,
        learning_rate=0.01,
        max_depth=8,
        min_child_weight=3,
        subsample=0.7,
        colsample_bytree=0.7,
        reg_alpha=0.1,
        reg_lambda=10,
        random_state=42,
        early_stopping_rounds=50,
        n_jobs=-1,
    )
}


📋 Modelos a comparar:
   1. Linear Regression (Baseline)
   2. Random Forest
   3. Gradient Boosting
   4. XGBoost (con configuración optimizada)


In [85]:
print("\n🔄 Entrenando y evaluando modelos...")
print("-" * 80)

all_results = []
training_times = {}

for name, model in models.items():
    print(f"\n▶ Entrenando {name}...")
    start_time = time.time()

    # Entrenar el modelo
    if name == 'XGBoost':
        # XGBoost con early stopping
        model.fit(
            X_train_transformed,
            y_train,
            eval_set=[(X_train_transformed, y_train), (X_valid_transformed, y_valid)],
            verbose=False
        )
    else:
        model.fit(X_train_transformed, y_train)

    training_time = time.time() - start_time
    training_times[name] = training_time

    print(f"  ✓ Entrenamiento completado en {training_time:.2f} segundos")

    # Generar predicciones
    y_train_pred = model.predict(X_train_transformed)
    y_valid_pred = model.predict(X_valid_transformed)
    y_test_pred = model.predict(X_test_transformed)

    # Calcular métricas para cada conjunto
    metrics_train = calculate_metrics(y_train, y_train_pred, f'{name} - Train')
    metrics_valid = calculate_metrics(y_valid, y_valid_pred, f'{name} - Valid')
    metrics_test = calculate_metrics(y_test, y_test_pred, f'{name} - Test')

    # Agregar a resultados
    all_results.extend([metrics_train, metrics_valid, metrics_test])

    # Mostrar métricas del modelo
    print(f"  📊 Valid RMSE: {metrics_valid['RMSE']:.2f} | R²: {metrics_valid['R²']:.4f}")



🔄 Entrenando y evaluando modelos...
--------------------------------------------------------------------------------

▶ Entrenando Linear Regression...
  ✓ Entrenamiento completado en 0.59 segundos
  📊 Valid RMSE: 6854.92 | R²: 0.8711

▶ Entrenando Random Forest...
  ✓ Entrenamiento completado en 14.36 segundos
  📊 Valid RMSE: 5989.52 | R²: 0.9016

▶ Entrenando Gradient Boosting...
  ✓ Entrenamiento completado en 24.98 segundos
  📊 Valid RMSE: 5926.33 | R²: 0.9037

▶ Entrenando XGBoost...
  ✓ Entrenamiento completado en 10.30 segundos
  📊 Valid RMSE: 5375.13 | R²: 0.9207


In [88]:
results_df = pd.DataFrame(all_results)

print("\n" + "="*80)
print("RESULTADOS COMPLETOS - TODOS LOS MODELOS")
print("="*80)
print(results_df.to_string(index=False))


RESULTADOS COMPLETOS - TODOS LOS MODELOS
                  Dataset        RMSE         MAE       R²    MAPE (%)
Linear Regression - Train 6436.355635 3418.073183 0.899096  471.261597
Linear Regression - Valid 6854.919129 3702.255801 0.871092 1032.782618
 Linear Regression - Test 4994.746077 2745.750607 0.936862  208.167958
    Random Forest - Train 4671.408672 2353.217441 0.946847  304.985515
    Random Forest - Valid 5989.519684 2874.336098 0.901585  856.128353
     Random Forest - Test 4066.122134 2043.884749 0.958157  108.974916
Gradient Boosting - Train 5051.883287 2499.444906 0.937837  329.199361
Gradient Boosting - Valid 5926.327191 2900.547601 0.903651  973.392429
 Gradient Boosting - Test 4091.273351 2058.502448 0.957638  125.699994
          XGBoost - Train 4288.780013 2242.310303 0.955198  273.476616
          XGBoost - Valid 5375.127161 2711.464355 0.920740  935.572639
           XGBoost - Test 3802.638295 2017.472168 0.963404  123.261358


In [89]:
print("\n" + "="*80)
print("COMPARACIÓN RESUMIDA (VALIDATION SET)")
print("="*80)

valid_results = results_df[results_df['Dataset'].str.contains('Valid')].copy()
valid_results['Modelo'] = valid_results['Dataset'].str.replace(' - Valid', '')
valid_results = valid_results[['Modelo', 'RMSE', 'MAE', 'R²', 'MAPE (%)']].sort_values('RMSE')

print(valid_results.to_string(index=False))



COMPARACIÓN RESUMIDA (VALIDATION SET)
           Modelo        RMSE         MAE       R²    MAPE (%)
          XGBoost 5375.127161 2711.464355 0.920740  935.572639
Gradient Boosting 5926.327191 2900.547601 0.903651  973.392429
    Random Forest 5989.519684 2874.336098 0.901585  856.128353
Linear Regression 6854.919129 3702.255801 0.871092 1032.782618


In [103]:
best_model_name = valid_results.iloc[0]['Modelo']
best_rmse = valid_results.iloc[0]['RMSE']
best_r2 = valid_results.iloc[0]['R²']

print(f"\n🏆 MEJOR MODELO: {best_model_name}")
print(f"   RMSE: {best_rmse:.2f}")
print(f"   R²: {best_r2:.4f}")
print(f"   Tiempo de entrenamiento: {training_times[best_model_name]:.2f} segundos")


🏆 MEJOR MODELO: XGBoost
   RMSE: 5375.13
   R²: 0.9207
   Tiempo de entrenamiento: 10.30 segundos


# CONCLUSIÓN DEL MODELO QUE VAMOS A USAR

Vamos a usar el modelo de RANDOM FOREST por el MAPE, ya que es muy inferior al de XGBOOST.

Si bien, el XGBoost tiene mejores RMSE y R2 (predice mejor casos donde el error es costoso) su MAPE es muy alto, por lo que falla mucho en errores típicos.

En cambio, Random Forest tiene un RMSE un poco menor, pero tiene mejor predicción ante errores típicos, por lo que se lo considera más "estable".

## POST AJUSTES OVERFITTING / UNDERFITTING

Después de pruebas de overfitting/underfitting se ajustaron campos como el max_depth del Random forest para que finalmente esté balanceado y además sea el mejor modelo. El XGBoost

In [104]:
print("\n" + "="*80)
print(f"DIAGNÓSTICO DE OVERFITTING - {best_model_name}")
print("="*80)

best_train_metrics = results_df[results_df['Dataset'] == f'{best_model_name} - Train'].iloc[0]
best_valid_metrics = results_df[results_df['Dataset'] == f'{best_model_name} - Valid'].iloc[0]
best_test_metrics = results_df[results_df['Dataset'] == f'{best_model_name} - Test'].iloc[0]

train_rmse = best_train_metrics['RMSE']
valid_rmse = best_valid_metrics['RMSE']
test_rmse = best_test_metrics['RMSE']

train_r2 = best_train_metrics['R²']
valid_r2 = best_valid_metrics['R²']
test_r2 = best_test_metrics['R²']

overfit_rmse = train_rmse - valid_rmse
overfit_r2 = valid_r2 - train_r2

print(f"📊 MÉTRICAS:")
print(f"   Train RMSE: {train_rmse:.2f} | R²: {train_r2:.4f}")
print(f"   Valid RMSE: {valid_rmse:.2f} | R²: {valid_r2:.4f}")
print(f"   Test  RMSE: {test_rmse:.2f}  | R²: {test_r2:.4f}")

print(f"\n🔍 ANÁLISIS:")
print(f"   Diferencia Train-Valid RMSE: {overfit_rmse:+.2f}")
print(f"   Diferencia Valid-Train R²: {overfit_r2:+.4f}")

# Diagnóstico automático
if overfit_rmse < -500:
    print("\n   ❌ OVERFITTING SEVERO")
    print("   💡 Soluciones:")
    print("      - Aumentar regularización")
    print("      - Reducir learning_rate")
    print("      - Reducir max_depth")
elif overfit_rmse < 0:
    print("\n   ⚠️ OVERFITTING MODERADO")
    print("   💡 Soluciones:")
    print("      - Aumentar ligeramente la regularización")
elif overfit_rmse < 200:
    print("\n   ✅ BIEN BALANCEADO")
elif overfit_rmse < 500:
    print("\n   ⚠️ LIGERO UNDERFITTING")
    print("   💡 Soluciones:")
    print("      - Aumentar max_depth")
    print("      - Aumentar learning_rate")
else:
    print("\n   ❌ UNDERFITTING SEVERO")
    print("   💡 Soluciones:")
    print("      - Aumentar complejidad del modelo")
    print("      - Reducir regularización")


DIAGNÓSTICO DE OVERFITTING - XGBoost
📊 MÉTRICAS:
   Train RMSE: 4288.78 | R²: 0.9552
   Valid RMSE: 5375.13 | R²: 0.9207
   Test  RMSE: 3802.64  | R²: 0.9634

🔍 ANÁLISIS:
   Diferencia Train-Valid RMSE: -1086.35
   Diferencia Valid-Train R²: -0.0345

   ❌ OVERFITTING SEVERO
   💡 Soluciones:
      - Aumentar regularización
      - Reducir learning_rate
      - Reducir max_depth


In [78]:
import os, json, joblib, sklearn
from sklearn.compose import make_column_selector

# IMPORTANTE: Guardar el modelo entrenado, no el modelo sin entrenar
# Los modelos ya fueron entrenados en el loop anterior, pero necesitamos guardar el mejor
# Re-entrenar el mejor modelo para asegurarnos de que esté entrenado
print(f"\n{'='*80}")
print("GUARDANDO ARTEFACTOS")
print(f"{'='*80}\n")

# Re-entrenar el mejor modelo con los datos completos
best_model = models[best_model_name]
if best_model_name == 'XGBoost':
    best_model.fit(
        X_train_transformed,
        y_train,
        eval_set=[(X_train_transformed, y_train), (X_valid_transformed, y_valid)],
        verbose=False
    )
else:
    best_model.fit(X_train_transformed, y_train)

ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

print("📦 Guardando fe_pipeline...")
joblib.dump(preprocessor, f"{ARTIFACT_DIR}/fe_pipeline.joblib")

print("📦 Guardando final_preprocessor...")
joblib.dump(final_pre, f"{ARTIFACT_DIR}/preprocessor.joblib")

print("📦 Guardando modelo...")
joblib.dump(best_model, f"{ARTIFACT_DIR}/model.joblib")

# Metadata
meta = {
    "model_name": best_model_name,
    "date_col": "fecha",
    "target_col": "cantidad",
    "requires_history": False,
    "dropped_cols": ["fecha", "provincia", "nombre_feriado", "tipo_transporte", "tipo_feriado", "linea", "empresa"],
    "winsorize_cols": ["t_med", "precip", "tmax", "tmin"],
    "sklearn_version": sklearn.__version__,
    "metrics": {
        "valid_rmse": best_rmse,
        "valid_r2": best_r2,
        "training_time_seconds": training_times[best_model_name]
    }
}
json.dump(meta, open(f"{ARTIFACT_DIR}/metadata.json", "w"), indent=2)

print(f"\n✅ Artefactos guardados exitosamente en '{ARTIFACT_DIR}/'")
print(f"\n📋 Archivos generados:")
print(f"   - fe_pipeline.joblib (feature engineering)")
print(f"   - preprocessor.joblib (encoding + scaling)")
print(f"   - model.joblib ({best_model_name})")
print(f"   - metadata.json (configuración)")

# Verificación: cargar y probar
print(f"\n🔍 Verificando carga de artefactos...")
fe_loaded = joblib.load(f"{ARTIFACT_DIR}/fe_pipeline.joblib")
prep_loaded = joblib.load(f"{ARTIFACT_DIR}/preprocessor.joblib")
model_loaded = joblib.load(f"{ARTIFACT_DIR}/model.joblib")
meta_loaded = json.load(open(f"{ARTIFACT_DIR}/metadata.json"))

print(f"   ✅ Feature engineering pipeline cargado")
print(f"   ✅ Preprocessor cargado")
print(f"   ✅ Modelo {meta_loaded['model_name']} cargado")
print(f"   ✅ Metadata cargada")

print(f"\n{'='*80}")
print("✅ EXPORTACIÓN COMPLETADA")
print(f"{'='*80}")


GUARDANDO ARTEFACTOS



KeyboardInterrupt: 

## 4. Ajuste de hiperparámetros



In [ ]:
param_distributions = {
    # Número de árboles: menos opciones, valores más razonables
    'n_estimators': [100, 150, 200],
    
    # Profundidad: enfocado en valores que eviten overfitting
    'max_depth': [8, 10, 12, 15],
    
    # Muestras por split: valores que balanceen bias-variance
    'min_samples_split': [5, 10, 15],
    
    # Muestras por hoja: control de overfitting
    'min_samples_leaf': [2, 4, 8],
    
    # Features por split: opciones estratégicas
    'max_features': ['sqrt', 0.5, 0.7],
    
    # Bootstrap: siempre True es más rápido y generalmente mejor
    'bootstrap': [True],
    
    # Porcentaje de muestras: enfocado en valores que mejoren generalización
    'max_samples': [0.7, 0.8, 0.9],
    
    # NUEVO: Controlar complejidad de árboles individuales
    'max_leaf_nodes': [None, 50, 100],
    
    # NUEVO: Regularización adicional
    'min_impurity_decrease': [0.0, 0.0001, 0.001]
}

print("\n📋 Espacio de búsqueda optimizado:")
print(f"   Total de combinaciones posibles: {np.prod([len(v) for v in param_distributions.values()])}")
for param, values in param_distributions.items():
    print(f"   - {param}: {len(values)} opciones")


📋 Espacio de búsqueda optimizado:
   Total de combinaciones posibles: 8748
   - n_estimators: 3 opciones
   - max_depth: 4 opciones
   - min_samples_split: 3 opciones
   - min_samples_leaf: 3 opciones
   - max_features: 3 opciones
   - bootstrap: 1 opciones
   - max_samples: 3 opciones
   - max_leaf_nodes: 3 opciones
   - min_impurity_decrease: 3 opciones


In [ ]:
print("\n🔍 Configurando búsqueda aleatoria optimizada...")

# CLAVE: Reducir n_iter pero mantener cv más pequeño para velocidad
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(
        random_state=42,
        n_jobs=-1,
        warm_start=False,  # No usar warm_start en búsqueda
        oob_score=True     # Usar out-of-bag score para evaluación adicional
    ),
    param_distributions=param_distributions,
    n_iter=20,              # REDUCIDO: 20 iteraciones en lugar de 30
    cv=3,                   # 3-fold es buen balance velocidad/precisión
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,              # Paralelizar todo
    random_state=42,
    verbose=1,              # Menos verbose para menos overhead
    return_train_score=True,
    refit=True              # Refit automático con mejores parámetros
)

print("✓ Configuración completada")
print(f"   - Iteraciones: 20 (balance velocidad/calidad)")
print(f"   - Cross-validation: 3 folds")
print(f"   - Total de fits: {20 * 3} = 60")
print(f"   - Tiempo estimado: 5-10 minutos")


🔍 Configurando búsqueda aleatoria optimizada...
✓ Configuración completada
   - Iteraciones: 20 (balance velocidad/calidad)
   - Cross-validation: 3 folds
   - Total de fits: 60 = 60
   - Tiempo estimado: 5-10 minutos


In [ ]:
print("\n🚀 Iniciando búsqueda de hiperparámetros...")
print("   (Esto tomará aproximadamente 5-10 minutos...)")

start_time = time.time()

random_search.fit(X_train_transformed, y_train)

search_time = time.time() - start_time

print(f"\n✅ Búsqueda completada en {search_time/60:.2f} minutos ({search_time:.2f} segundos)")



🚀 Iniciando búsqueda de hiperparámetros...
   (Esto tomará aproximadamente 5-10 minutos...)
Fitting 3 folds for each of 20 candidates, totalling 60 fits



✅ Búsqueda completada en 6.52 minutos (390.91 segundos)


In [ ]:
# ============================================================================
# ANÁLISIS DE RESULTADOS
# ============================================================================

print("\n" + "="*80)
print("RESULTADOS DE LA BÚSQUEDA")
print("="*80)

print("\n🏆 MEJORES HIPERPARÁMETROS ENCONTRADOS:")
for param, value in random_search.best_params_.items():
    print(f"   {param}: {value}")

# Score del mejor modelo
best_cv_rmse = -random_search.best_score_
print(f"\n📊 Mejor RMSE en Cross-Validation: {best_cv_rmse:.2f}")

# Información del mejor estimador
best_model = random_search.best_estimator_
if hasattr(best_model, 'oob_score_'):
    oob_rmse = np.sqrt(-best_model.oob_score_) if best_model.oob_score_ < 0 else np.sqrt(1 - best_model.oob_score_)
    print(f"📊 OOB Score del mejor modelo: {best_model.oob_score_:.4f}")



RESULTADOS DE LA BÚSQUEDA

🏆 MEJORES HIPERPARÁMETROS ENCONTRADOS:
   n_estimators: 150
   min_samples_split: 15
   min_samples_leaf: 2
   min_impurity_decrease: 0.0
   max_samples: 0.8
   max_leaf_nodes: None
   max_features: 0.5
   max_depth: 12
   bootstrap: True

📊 Mejor RMSE en Cross-Validation: 3125.38
📊 OOB Score del mejor modelo: 0.9803


In [ ]:
print("\n" + "="*80)
print("ENTRENAMIENTO DEL MODELO OPTIMIZADO")
print("="*80)

print("\n🔄 Entrenando Random Forest con hiperparámetros optimizados...")

final_rf_model = RandomForestRegressor(
    **random_search.best_params_,
    random_state=42,
    n_jobs=-1
)

start_train = time.time()
final_rf_model.fit(X_train_transformed, y_train)
train_time = time.time() - start_train

print(f"✓ Modelo entrenado en {train_time:.2f} segundos")


ENTRENAMIENTO DEL MODELO OPTIMIZADO

🔄 Entrenando Random Forest con hiperparámetros optimizados...
✓ Modelo entrenado en 9.93 segundos


In [ ]:
print("\n📊 Generando predicciones...")

y_train_pred_opt = final_rf_model.predict(X_train_transformed)
y_valid_pred_opt = final_rf_model.predict(X_valid_transformed)
y_test_pred_opt = final_rf_model.predict(X_test_transformed)

# Calcular métricas
metrics_train_opt = calculate_metrics(y_train, y_train_pred_opt, 'Train (Optimizado)')
metrics_valid_opt = calculate_metrics(y_valid, y_valid_pred_opt, 'Valid (Optimizado)')
metrics_test_opt = calculate_metrics(y_test, y_test_pred_opt, 'Test (Optimizado)')

results_opt_df = pd.DataFrame([metrics_train_opt, metrics_valid_opt, metrics_test_opt])

print("\n" + "="*80)
print("MÉTRICAS DEL MODELO OPTIMIZADO")
print("="*80)
print(results_opt_df.to_string(index=False))


📊 Generando predicciones...

MÉTRICAS DEL MODELO OPTIMIZADO
           Dataset        RMSE         MAE       R²    MAPE (%)
Train (Optimizado) 2432.196121 1132.818424 0.985591  132.584015
Valid (Optimizado) 3776.322544 1564.476856 0.960879 1211.014633
 Test (Optimizado) 2758.393517 1195.714962 0.980744   46.913720


In [ ]:
print("\n" + "="*80)
print("COMPARACIÓN: MODELO ANTERIOR vs MODELO OPTIMIZADO")
print("="*80)

comparison_data = {
    'Conjunto': ['Train', 'Valid', 'Test'],
    'RMSE Anterior': [
        best_train_metrics['RMSE'],
        modelo_anterior_valid_rmse,
        modelo_anterior_test_rmse
    ],
    'RMSE Optimizado': [
        metrics_train_opt['RMSE'],
        metrics_valid_opt['RMSE'],
        metrics_test_opt['RMSE']
    ],
    'R² Anterior': [
        best_train_metrics['R²'],
        modelo_anterior_valid_r2,
        modelo_anterior_test_r2
    ],
    'R² Optimizado': [
        metrics_train_opt['R²'],
        metrics_valid_opt['R²'],
        metrics_test_opt['R²']
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df['Mejora RMSE'] = comparison_df['RMSE Anterior'] - comparison_df['RMSE Optimizado']
comparison_df['Mejora R²'] = comparison_df['R² Optimizado'] - comparison_df['R² Anterior']

print("\n" + comparison_df.to_string(index=False))


COMPARACIÓN: MODELO ANTERIOR vs MODELO OPTIMIZADO

Conjunto  RMSE Anterior  RMSE Optimizado  R² Anterior  R² Optimizado  Mejora RMSE  Mejora R²
   Train    1950.085575      2432.196121     0.990737       0.985591  -482.110546  -0.005146
   Valid    3666.564332      3776.322544     0.963120       0.960879  -109.758212  -0.002241
    Test    2681.070402      2758.393517     0.981808       0.980744   -77.323115  -0.001064


In [ ]:
print("\n" + "="*80)
print("ANÁLISIS DE MEJORAS")
print("="*80)

mejora_rmse_valid = modelo_anterior_valid_rmse - metrics_valid_opt['RMSE']
mejora_r2_valid = metrics_valid_opt['R²'] - modelo_anterior_valid_r2
mejora_porcentual = (mejora_rmse_valid / modelo_anterior_valid_rmse) * 100

print(f"\n📈 MEJORAS EN VALIDATION SET:")
print(f"   Mejora RMSE: {mejora_rmse_valid:+.2f} ({mejora_porcentual:+.2f}%)")
print(f"   Mejora R²: {mejora_r2_valid:+.4f}")

if mejora_rmse_valid > 50:
    print("\n   🏆 EXCELENTE: Mejora significativa con la optimización")
elif mejora_rmse_valid > 10:
    print("\n   ✅ BUENO: Mejora notable con la optimización")
elif mejora_rmse_valid > 0:
    print("\n   ✓ POSITIVO: Ligera mejora con la optimización")
else:
    print("\n   ⚠️ El modelo anterior era mejor o similar")
    print("   💡 Considerar:")
    print("      - El modelo base ya estaba bien ajustado")
    print("      - Posible overfitting en la búsqueda de hiperparámetros")

# Análisis de overfitting
overfit_rmse_opt = metrics_train_opt['RMSE'] - metrics_valid_opt['RMSE']

print(f"\n🔍 DIAGNÓSTICO DE OVERFITTING:")
print(f"   Diferencia Train-Valid RMSE: {overfit_rmse_opt:+.2f}")

if abs(overfit_rmse_opt) < 200:
    print("   ✅ BIEN BALANCEADO: Modelo optimizado bien generalizado")
elif overfit_rmse_opt < 0:
    print("   ⚠️ OVERFITTING DETECTADO: El modelo memoriza datos de train")
else:
    print("   ⚠️ UNDERFITTING LEVE: El modelo podría ser más complejo")

# Consistencia Valid-Test
valid_test_diff = abs(metrics_valid_opt['RMSE'] - metrics_test_opt['RMSE'])
print(f"\n📊 CONSISTENCIA VALID-TEST:")
print(f"   Diferencia RMSE: {valid_test_diff:.2f}")

if valid_test_diff < 300:
    print("   ✅ EXCELENTE: Performance consistente")
elif valid_test_diff < 500:
    print("   ✅ BUENO: Performance aceptable")
else:
    print("   ⚠️ Hay diferencia significativa entre Valid y Test")



ANÁLISIS DE MEJORAS

📈 MEJORAS EN VALIDATION SET:
   Mejora RMSE: -74.78 (-0.41%)
   Mejora R²: -0.0076

   ⚠️ El modelo anterior era mejor o similar
   💡 Considerar:
      - El modelo base ya estaba bien ajustado
      - Posible overfitting en la búsqueda de hiperparámetros

🔍 DIAGNÓSTICO DE OVERFITTING:
   Diferencia Train-Valid RMSE: +410.06
   ⚠️ UNDERFITTING LEVE: El modelo podría ser más complejo

📊 CONSISTENCIA VALID-TEST:
   Diferencia RMSE: 822.54
   ⚠️ Hay diferencia significativa entre Valid y Test


## 6. Predicción final (ejemplo de una fecha futura)

In [ ]:
## 6. Predicción Final - Ejemplo Práctico

import pandas as pd
import numpy as np

print("="*80)
print("PREDICCIÓN FINAL - EJEMPLO PRÁCTICO")
print("="*80)

# ----------------------------------------------------------------------------
# 6.1 Ejemplo con datos reales del Test Set
# ----------------------------------------------------------------------------

print("\n📋 EJEMPLO 1: PREDICCIÓN CON DATOS REALES DEL TEST SET")
print("-" * 80)

# Tomar una muestra aleatoria del test set
np.random.seed(42)
sample_idx = np.random.randint(0, len(X_test))
sample_data_raw = X_test.iloc[sample_idx:sample_idx+1].copy()

# Obtener fecha correspondiente del dataframe del test set
sample_date_info = df_test.iloc[sample_idx]

print("\n📅 INFORMACIÓN DE LA PREDICCIÓN:")
print(f"   Fecha: {sample_date_info['fecha'].strftime('%Y-%m-%d (%A)')}")
print(f"   Línea: {sample_data_raw['linea'].iloc[0]}")
print(f"   Municipio: {sample_data_raw['municipio'].iloc[0]}")
print(f"   Empresa: {sample_data_raw['empresa'].iloc[0]}")

print(f"\n🌤️ CONDICIONES CLIMÁTICAS:")
print(f"   Temperatura máxima: {sample_data_raw['tmax'].iloc[0]:.1f}°C")
print(f"   Temperatura mínima: {sample_data_raw['tmin'].iloc[0]:.1f}°C")
print(f"   Temperatura media: {sample_data_raw['t_med'].iloc[0]:.1f}°C")
print(f"   Precipitación: {sample_data_raw['precip'].iloc[0]:.1f} mm")
print(f"   Viento: {sample_data_raw['viento'].iloc[0]:.1f} km/h")

print(f"\n📊 CONTEXTO TEMPORAL:")
dia_nombres = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
dia_semana_num = int(sample_data_raw['dia_semana'].iloc[0])
print(f"   Día de la semana: {dia_nombres[dia_semana_num]}")
print(f"   Es fin de semana: {'Sí' if sample_data_raw['is_weekend'].iloc[0] else 'No'}")
print(f"   Mes: {sample_data_raw['mes'].iloc[0]}")
print(f"   Es feriado: {'Sí' if sample_data_raw['is_feriado'].iloc[0] else 'No'}")

print(f"\n📊 PERFILES HISTÓRICOS:")
print("   (Los perfiles históricos se calculan automáticamente durante la predicción)")
print("   El modelo utiliza estadísticas históricas de pasajeros por:")
print("   - Línea + Municipio + Día de la semana")
print("   - Línea + Municipio")
print("   - Línea + Día de la semana")
print("   - Municipio + Día de la semana")

# Preprocesar y predecir (usar ambos pipelines)
sample_data_fe = preprocessor.transform(sample_data_raw)
sample_data_transformed = final_pre.transform(sample_data_fe)
prediction = final_rf_model.predict(sample_data_transformed)[0]
actual = y_test.iloc[sample_idx]

print(f"\n🎯 RESULTADO DE LA PREDICCIÓN:")
print(f"   Predicción del modelo: {prediction:.0f} pasajeros")
print(f"   Valor real observado: {actual:.0f} pasajeros")
print(f"   Error absoluto: {abs(prediction - actual):.0f} pasajeros")

# Evaluación cualitativa del error
error_pct = abs(prediction - actual) / actual * 100
if error_pct < 5:
    print("   ✅ Excelente: Error menor al 5%")
elif error_pct < 10:
    print("   ✅ Muy bueno: Error menor al 10%")
elif error_pct < 20:
    print("   ✓ Aceptable: Error menor al 20%")
else:
    print("   ⚠️ Alto: Error mayor al 20%")

# ----------------------------------------------------------------------------
# 6.2 Función para predicciones futuras
# ----------------------------------------------------------------------------

print("\n" + "="*80)
print("FUNCIÓN DE PREDICCIÓN FUTURA")
print("="*80)

def predict_future_passengers(linea, municipio, empresa, fecha, tmax, tmin, precip, viento,
                              lag_1=None, lag_7=None, lag_28=None, is_feriado=0, tipo_feriado=None):
    """
    Predice la cantidad de pasajeros para una fecha futura.

    Parámetros:
    -----------
    linea : str
        Código de la línea de colectivo (ej: 'BSAS_LINEA_740')
    municipio : str
        Nombre del municipio (ej: 'san miguel')
    empresa : str
        Nombre de la empresa operadora
    fecha : str
        Fecha en formato 'YYYY-MM-DD'
    tmax : float
        Temperatura máxima esperada (°C)
    tmin : float
        Temperatura mínima esperada (°C)
    precip : float
        Precipitación esperada (mm)
    viento : float
        Velocidad del viento esperada (km/h)
    lag_1 : float, optional
        Pasajeros del día anterior (si no se provee, usa promedio histórico)
    lag_7 : float, optional
        Pasajeros hace 7 días (si no se provee, usa promedio histórico)
    lag_28 : float, optional
        Pasajeros hace 28 días (si no se provee, usa promedio histórico)
    is_feriado : int, optional
        1 si es feriado, 0 si no lo es (default: 0)
    tipo_feriado : str, optional
        Tipo de feriado si corresponde

    Retorna:
    --------
    float
        Cantidad estimada de pasajeros
    """


    # ESTO SE EVITA CON EL PIPELINE DE LA PARTE DEL PRINCIPIO!!!
    # Convertir fecha a datetime
    fecha_dt = pd.to_datetime(fecha)

    # Calcular features temporales
    dia_semana = fecha_dt.dayofweek
    mes = fecha_dt.month
    is_weekend = 1 if dia_semana >= 5 else 0

    # Features cíclicas
    mes_sin = np.sin(2 * np.pi * mes / 12)
    mes_cos = np.cos(2 * np.pi * mes / 12)
    dia_sin = np.sin(2 * np.pi * dia_semana / 7)
    dia_cos = np.cos(2 * np.pi * dia_semana / 7)

    # Features de temperatura
    t_med = (tmax + tmin) / 2
    t_amp = tmax - tmin

    # Valores por defecto para lags si no se proveen
    # Usar promedio general del training set
    if lag_1 is None:
        lag_1 = y_train.mean()
    if lag_7 is None:
        lag_7 = y_train.mean()
    if lag_28 is None:
        lag_28 = y_train.mean()

    roll_7 = lag_1  # Aproximación
    roll_28 = lag_7  # Aproximación

    # Crear DataFrame con todos los features
    future_data = pd.DataFrame({
        'empresa': [empresa],
        'linea': [linea],
        'municipio': [municipio],
        'tmax': [tmax],
        'tmin': [tmin],
        'precip': [precip],
        'viento': [viento],
        'is_feriado': [is_feriado],
        'tipo_feriado': [tipo_feriado],
        'dia_semana': [dia_semana],
        'mes': [mes],
        'is_weekend': [is_weekend],
        'mes_sin': [mes_sin],
        'mes_cos': [mes_cos],
        'dia_sin': [dia_sin],
        'dia_cos': [dia_cos],
        't_med': [t_med],
        't_amp': [t_amp],
        'lag_1': [lag_1],
        'lag_7': [lag_7],
        'lag_28': [lag_28],
        'roll_7': [roll_7],
        'roll_28': [roll_28],
        'has_lag_1': [0],
        'has_lag_7': [0],
        'has_lag_28': [0]
    })

      # Aplicar feature engineering primero
    future_data_fe = preprocessor.transform(future_data)
      # Luego aplicar preprocesamiento final
    future_data_transformed = final_pre.transform(future_data_fe)
    prediction = final_rf_model.predict(future_data_transformed)[0]

    return prediction

print("\n✅ Función 'predict_future_passengers' creada")
print("\n📝 Parámetros de la función:")
print("   - linea: Código de línea (obligatorio)")
print("   - municipio: Municipio (obligatorio)")
print("   - empresa: Empresa operadora (obligatorio)")
print("   - fecha: Fecha YYYY-MM-DD (obligatorio)")
print("   - tmax, tmin: Temperaturas en °C (obligatorio)")
print("   - precip: Precipitación en mm (obligatorio)")
print("   - viento: Velocidad en km/h (obligatorio)")
print("   - lag_1, lag_7, lag_28: Datos históricos (opcional)")
print("   - is_feriado: 0 o 1 (opcional, default: 0)")

# ----------------------------------------------------------------------------
# 6.3 Ejemplos de uso de la función
# ----------------------------------------------------------------------------

print("\n" + "="*80)
print("EJEMPLOS DE PREDICCIONES FUTURAS")
print("="*80)

# Ejemplo 1: Día laboral normal
print("\n📝 EJEMPLO 1: DÍA LABORAL NORMAL (Miércoles)")
print("-" * 80)

pred_ejemplo1 = predict_future_passengers(
    linea="BSAS_LINEA_740",
    municipio="san miguel",
    empresa="LA PRIMERA DE GRAND BOURG S.A.",
    fecha="2025-01-15",  # Miércoles
    tmax=28.0,
    tmin=20.0,
    precip=0.0,
    viento=15.0,
    lag_1=15000,  # Ayer hubo 15,000 pasajeros
    lag_7=14500,  # Hace una semana hubo 14,500
    lag_28=14000, # Hace un mes hubo 14,000
    is_feriado=0
)

print(f"   Fecha: 2025-01-15 (Miércoles)")
print(f"   Condiciones: Buen clima, sin lluvia")
print(f"   Predicción: {pred_ejemplo1:.0f} pasajeros")

# Ejemplo 2: Fin de semana
print("\n📝 EJEMPLO 2: FIN DE SEMANA (Domingo)")
print("-" * 80)

pred_ejemplo2 = predict_future_passengers(
    linea="BSAS_LINEA_740",
    municipio="san miguel",
    empresa="LA PRIMERA DE GRAND BOURG S.A.",
    fecha="2025-01-19",  # Domingo
    tmax=30.0,
    tmin=22.0,
    precip=0.0,
    viento=10.0,
    lag_1=8000,   # Los domingos hay menos pasajeros
    lag_7=14500,
    lag_28=14000,
    is_feriado=0
)

print(f"   Fecha: 2025-01-19 (Domingo)")
print(f"   Condiciones: Día soleado")
print(f"   Predicción: {pred_ejemplo2:.0f} pasajeros")
print(f"   Nota: Menor que día laboral (esperado para fin de semana)")

# Ejemplo 3: Día con lluvia
print("\n📝 EJEMPLO 3: DÍA LLUVIOSO (Lunes)")
print("-" * 80)

pred_ejemplo3 = predict_future_passengers(
    linea="BSAS_LINEA_740",
    municipio="san miguel",
    empresa="LA PRIMERA DE GRAND BOURG S.A.",
    fecha="2025-01-20",  # Lunes
    tmax=22.0,
    tmin=18.0,
    precip=25.0,  # Lluvia intensa
    viento=30.0,  # Viento fuerte
    lag_1=8000,   # Domingo (menos pasajeros)
    lag_7=14500,
    lag_28=14000,
    is_feriado=0
)

print(f"   Fecha: 2025-01-20 (Lunes)")
print(f"   Condiciones: Lluvia intensa, viento fuerte")
print(f"   Predicción: {pred_ejemplo3:.0f} pasajeros")
print(f"   Nota: El clima puede afectar la demanda")

# Ejemplo 4: Feriado
print("\n📝 EJEMPLO 4: DÍA FERIADO")
print("-" * 80)

pred_ejemplo4 = predict_future_passengers(
    linea="BSAS_LINEA_740",
    municipio="san miguel",
    empresa="LA PRIMERA DE GRAND BOURG S.A.",
    fecha="2025-05-01",  # 1 de Mayo (Día del Trabajador)
    tmax=25.0,
    tmin=18.0,
    precip=0.0,
    viento=12.0,
    lag_1=12000,
    lag_7=14000,
    lag_28=14500,
    is_feriado=1,  # Es feriado
    tipo_feriado="inamovible"
)

print(f"   Fecha: 2025-05-01 (Día del Trabajador)")
print(f"   Condiciones: Feriado nacional")
print(f"   Predicción: {pred_ejemplo4:.0f} pasajeros")
print(f"   Nota: Los feriados típicamente tienen menos demanda")

# Ejemplo 5: Sin datos históricos (usando promedios)
print("\n📝 EJEMPLO 5: SIN DATOS HISTÓRICOS (Nueva ruta o predicción lejana)")
print("-" * 80)

pred_ejemplo5 = predict_future_passengers(
    linea="BSAS_LINEA_740",
    municipio="san miguel",
    empresa="LA PRIMERA DE GRAND BOURG S.A.",
    fecha="2025-06-15",  # Fecha lejana
    tmax=20.0,
    tmin=12.0,
    precip=0.0,
    viento=18.0,
    # No se proveen lags, usa promedios automáticamente
    is_feriado=0
)

print(f"   Fecha: 2025-06-15")
print(f"   Condiciones: Sin datos históricos recientes")
print(f"   Predicción: {pred_ejemplo5:.0f} pasajeros")
print(f"   Nota: Basado en promedios históricos generales")

# ----------------------------------------------------------------------------
# 6.4 Resumen y recomendaciones
# ----------------------------------------------------------------------------

print("\n" + "="*80)
print("RESUMEN Y RECOMENDACIONES")
print("="*80)

print("\n💡 CÓMO USAR LA FUNCIÓN DE PREDICCIÓN:")
print("   1. Preparar datos de entrada (línea, municipio, empresa)")
print("   2. Obtener pronóstico climático para la fecha")
print("   3. Si es posible, incluir datos históricos recientes (lags)")
print("   4. Indicar si la fecha es feriado")
print("   5. La función retorna la cantidad estimada de pasajeros")

print("\n⚠️ CONSIDERACIONES IMPORTANTES:")
print("   - Las predicciones son más precisas con datos históricos recientes")
print("   - El clima puede afectar significativamente la demanda")
print("   - Los fines de semana y feriados tienen patrones diferentes")
print("   - Para rutas nuevas, la precisión será menor")

print("\n✅ MODELO LISTO PARA PRODUCCIÓN")
print("   El modelo ha sido entrenado, validado y está listo para:")
print("   - Predicciones en tiempo real")
print("   - Planificación de servicios")
print("   - Optimización de frecuencias")
print("   - Análisis de demanda futura")

PREDICCIÓN FINAL - EJEMPLO PRÁCTICO

📋 EJEMPLO 1: PREDICCIÓN CON DATOS REALES DEL TEST SET
--------------------------------------------------------------------------------

📅 INFORMACIÓN DE LA PREDICCIÓN:
   Fecha: 2024-11-28 (Thursday)
   Línea: BSAS_LINEA_506
   Municipio: florencio varela
   Empresa: TRANSPORTES SAN JUAN BAUTISTA S.A.

🌤️ CONDICIONES CLIMÁTICAS:
   Temperatura máxima: 27.3°C
   Temperatura mínima: 17.7°C
   Temperatura media: 22.5°C
   Precipitación: 1.0 mm
   Viento: 22.9 km/h

📊 CONTEXTO TEMPORAL:
   Día de la semana: Jueves
   Es fin de semana: No
   Mes: 11
   Es feriado: No

📊 PERFILES HISTÓRICOS:
   (Los perfiles históricos se calculan automáticamente durante la predicción)
   El modelo utiliza estadísticas históricas de pasajeros por:
   - Línea + Municipio + Día de la semana
   - Línea + Municipio
   - Línea + Día de la semana
   - Municipio + Día de la semana


KeyError: 'std'

## 7. Conclusiones del trabajo